In [1]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect()
%sql duckdb:///:memory:
%sql conn --alias duckdb

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Connecting to 'duckdb:///:memory:'

In [2]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RidersSelectedBy   AS FROM "../data/riders_selected_by.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

,Count
0,14


In [3]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Stage_ID
    ,COALESCE(results.Stage_Date, races.RaceStart) AS Stage_Date
    ,results.Stage_Type
    ,CASE WHEN results.Stage_ID <> 'gc' AND (results.Stage_Type <> 'TTT' OR results.Stage_Type IS NULL) THEN 1 ELSE 0 END AS Løbsdage
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,riders.RiderPrice
    ,riders.RiderTeam
    ,COALESCE(selected.ValgtAf, 0) AS SelectedBy
    ,CASE WHEN cbans.RiderName IS NULL AND riders.RiderPrice <= 2.5 THEN 'Ja' ELSE '' END AS Cheapo
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Riders riders
LEFT JOIN Results_2025 results ON results.Rider = riders.RiderName_PCS
LEFT JOIN Races races ON races.RaceName_PCS = results.Race
LEFT JOIN CheapoBans cbans ON cbans.RiderName = riders.RiderName_Zweeler
LEFT JOIN RidersSelectedBy selected ON selected.RiderName_Zweeler = riders.RiderName_Zweeler
LEFT JOIN PointsSystem points ON 1 = 1
    AND (results.Stage_Type IS NULL OR results.Stage_Type <> 'TTT')
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_ID LIKE '%stage%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory;

,Count
0,14001


In [4]:
%%sql
rider_output << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeams
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(Points) AS int), 0) AS Point
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     *
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src;

In [5]:
%%sql
rider_output_cheapo << 
WITH
ManagerLists AS
(
    SELECT RiderName, STRING_AGG(ManagerName, ', ' ORDER BY ManagerName ASC) AS Managers
    FROM ManagerTeamsCheapo
    GROUP BY RiderName
),
RiderOverview AS 
(
    SELECT
         RiderName_Zweeler AS Navn
        ,MAX(RiderTeam) AS Hold
        ,MAX(RiderPrice) AS Pris
        ,MAX(SelectedBy) AS Valgt
        ,CAST(SUM(Løbsdage) AS int) AS Løbsdage
        ,COALESCE(CAST(SUM(CheapoPoints) AS int), 0) AS Point
        ,MAX(Cheapo) AS Cheapo
    FROM RaceResultsPoints
    GROUP BY RiderName_Zweeler
)
SELECT 
     Navn
    ,Hold
    ,Pris
    ,Løbsdage
    ,Point
    ,ROUND(Point / Pris, 1) AS "P/mil"
    ,ROUND(CASE WHEN Løbsdage = 0 THEN 0 ELSE Point / Løbsdage END, 1) AS "P/dage"
    ,COALESCE((SELECT Managers FROM ManagerLists sub WHERE sub.RiderName = src.Navn), '') AS Managers
FROM RiderOverview src
WHERE Cheapo = 'Ja';

In [6]:
%%sql
WITH Src AS
(
    SELECT YEARWEEK(results.Stage_Date) AS Uge, teams.ManagerName AS Manager, CAST(SUM(Points) AS int) AS Point
    FROM RaceResultsPoints results
    INNER JOIN ManagerTeams teams ON teams.RiderName = results.RiderName_Zweeler
    WHERE 1 = 1
        AND Uge IS NOT NULL
    GROUP BY YEARWEEK(results.Stage_Date), teams.ManagerName
)
PIVOT Src
ON Uge
USING COALESCE(SUM(Point)::int, 0)
GROUP BY Manager
ORDER BY Manager;

,Manager,202504,202505,202506,202507,202508,202509,202510,202511,202512,202513,202514,202515,202516,202517
0,Chrelle,312,100,67,130,653,72,280,404,125,266,420,572,293,467
1,Hustlersen,484,26,6,188,437,0,161,227,127,623,304,258,144,290
2,Jappo,156,130,15,130,599,139,379,405,141,547,378,295,207,520
3,Jarma,8,105,29,38,557,179,207,284,205,356,394,525,258,339
4,Kenk,283,49,106,94,539,110,225,428,167,570,316,464,118,340
5,Knak,94,146,15,35,540,96,344,273,153,418,191,400,220,545
6,Matti,54,36,214,123,595,63,27,344,118,511,304,355,264,284
7,Okholm,183,68,49,22,636,109,162,481,101,361,260,328,200,345
8,Tommy,189,94,27,88,414,138,219,269,160,390,454,375,338,322
9,Visti,56,106,77,132,425,110,193,378,192,573,311,308,158,379


In [7]:
rider_output.to_html("../outputtables/RytteroversigtTabel.html", table_id = "filterabletable", index = False)
rider_output_cheapo.to_html("../outputtables/RytteroversigtCheapoTabel.html", table_id = "filterabletable", index = False)